In [1]:
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
import re
import scipy as sp

plt.style.use('fivethirtyeight')

# 노트북 안에 그래프 그리기
%matplotlib inline
# 그래프 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일 사용
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus']=False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

# 한글 font 설정
import matplotlib.font_manager as fm

font_name = fm.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
plt.rc('font',family = font_name)
mpl.rcParams['axes.unicode_minus'] = False

# final_data.csv 고려사항

    - 성별, 연령대 결측치 처리: 상품 소분류에서 성별, 연령대 최빈값으로 변경
    
    - 검색을 통해 구입된 데이터들만 포함
    
    - 검색어에 공백 포함된 데이터값 있음
    
    - MEMBER = 회원:0, 비회원:1
    
    - search_purpose = search_purpose_index를 이용해 목적 구매자와 충동 구매자를 1과 0으로 나눈 컬럼
      : 검색어 vs [브랜드, 상품, 추가정보, 대분류, 중분류, 소분류]
      : 각각 문장유사도 분석했을 때 모든 값이 0이 나와야 충동구매자로 판단
      : 검색어가 각 컬럼과 조금이라도 연관성이 있다면 목적성이 있다

# 챗봇 입력 변수

    - 성별, 연령대
    
    - 날씨?: 외부 데이터(크롤링 필요)

In [16]:
final_data = pd.read_csv("final_data.csv")
pmcs = pd.read_csv("PMCS.csv")

In [3]:
final_data

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK,KWD_NM,SEARCH_CNT,search_purpose,search_purpose_index
0,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3.0,0,0.000000
1,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3.0,0,0.000000
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1.0,1,0.285714
3,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23.0,1,1.000000
4,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3.0,0,0.000000
5,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3.0,1,2.500000
6,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1.0,0,0.000000
7,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23.0,0,0.000000
8,4142395,6158159,43,176911,notset,아베다,90000,1,6월스무드인퓨전샴푸리터세트,퍼스널케어,헤어케어,린스/컨디셔너,F,40.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3.0,0,0.000000
9,4142395,6158159,43,176911,notset,아베다,90000,1,6월스무드인퓨전샴푸리터세트,퍼스널케어,헤어케어,린스/컨디셔너,F,40.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3.0,0,0.000000


### think1. 목적 구매자 vs 충동 구매자

    - CLNT_ID와 SESS_ID를 기준으로 그룹화 했을 때, search_purpose값이 0.5 이하면 충동구매자로 판단

In [4]:
df = final_data.groupby(["CLNT_ID", "SESS_ID"])[["search_purpose"]].mean().reset_index()
df

,CLNT_ID,SESS_ID,search_purpose
0,14,1156991,0.250000
1,25,5317297,0.000000
2,25,6059256,1.000000
3,25,8113243,0.500000
4,25,9506206,1.000000
5,33,8694850,0.500000
6,33,10300429,0.000000
7,33,10548225,0.166667
8,56,2108568,1.000000
9,56,5747479,0.000000


In [5]:
no_p_clnt = df[df["search_purpose"]<=0.5]["CLNT_ID"]  #> 517462
no_p_clnt.count()

517462

In [6]:
yes_p_clnt = df[df["search_purpose"]>0.5]["CLNT_ID"]  #> 704618
yes_p_clnt.count()

704618

In [7]:
final_data[final_data["search_purpose"]==0].count()  #> 3146433

CLNT_ID                 3146433
SESS_ID                 3146433
HITS_SEQ                3146433
PD_C                    3146433
PD_ADD_NM               3146433
PD_BRA_NM               3146433
PD_BUY_AM               3146433
PD_BUY_CT               3146433
PD_NM                   3146433
CLAC1_NM                3146433
CLAC2_NM                3146433
CLAC3_NM                3146433
CLNT_GENDER             3146433
CLNT_AGE                3146433
MEMBER                  3146433
SESS_SEQ                3146433
SESS_DT                 3146433
TOT_PAG_VIEW_CT         3146433
TOT_SESS_HR_V           3146433
DVC_CTG_NM              3146433
ZON_NM                  3146433
CITY_NM                 3146433
SESS_MONTH              3146433
SESS_DAY                3146433
SESS_WEEK               3146433
KWD_NM                  3146433
SEARCH_CNT              3146433
search_purpose          3146433
search_purpose_index    3146433
dtype: int64

In [8]:
final_data[final_data["search_purpose"]==1].count()  #> 3291794

CLNT_ID                 3291794
SESS_ID                 3291794
HITS_SEQ                3291794
PD_C                    3291794
PD_ADD_NM               3291793
PD_BRA_NM               3291794
PD_BUY_AM               3291794
PD_BUY_CT               3291794
PD_NM                   3291794
CLAC1_NM                3291794
CLAC2_NM                3291794
CLAC3_NM                3291794
CLNT_GENDER             3291794
CLNT_AGE                3291794
MEMBER                  3291794
SESS_SEQ                3291794
SESS_DT                 3291794
TOT_PAG_VIEW_CT         3291794
TOT_SESS_HR_V           3291794
DVC_CTG_NM              3291794
ZON_NM                  3291794
CITY_NM                 3291794
SESS_MONTH              3291794
SESS_DAY                3291794
SESS_WEEK               3291794
KWD_NM                  3291794
SEARCH_CNT              3291794
search_purpose          3291794
search_purpose_index    3291794
dtype: int64

### think2. 상품 하나 구매하는데 본 세션 페이지

    - 가설: 목적 구매자가 충동 구매자보다 상품 하나 구매하며 본 세션 페이지가 많을 것이다
    
    - search_purpose 0과 1로 분류해서 평균 비교시, 충동 구매자 세션 페이지 조회 수가 약 27정도 높게 나타남
    
    - CLNT_ID, SESS_ID로 그룹화하여 search_purpose 0.5 기준으로 나누어 평균 비교시, 충동 구매자 세션 페이지 조회 수가 약 31 정도 높게 나타남

In [14]:
test = final_data

In [20]:
avg_view = test.groupby(["CLNT_ID", "SESS_ID"])["PD_BUY_CT", "TOT_PAG_VIEW_CT"].mean().reset_index()
avg_view

avg_view["avg_PAG_VIEW_CT"] = avg_view["TOT_PAG_VIEW_CT"]/avg_view["PD_BUY_CT"]
avg_view

test = pd.merge(test, avg_view, on=["CLNT_ID", "SESS_ID"], how="left")
test.isnull().sum()

#test.drop(["PD_BUY_CT_y", "TOT_PAG_VIEW_CT_y"], axis=1, inplace=True)

,CLNT_ID,SESS_ID,PD_BUY_CT,TOT_PAG_VIEW_CT
0,14,1156991,1.000000,256.0
1,25,5317297,1.000000,33.0
2,25,6059256,1.000000,7.0
3,25,8113243,1.000000,28.0
4,25,9506206,1.000000,12.0
5,33,8694850,1.000000,20.0
6,33,10300429,1.000000,27.0
7,33,10548225,1.000000,48.0
8,56,2108568,1.000000,44.0
9,56,5747479,1.000000,49.0


In [28]:
test.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT_x,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_GENDER,CLNT_AGE,MEMBER,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT_x,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,SESS_MONTH,SESS_DAY,SESS_WEEK,KWD_NM,SEARCH_CNT,search_purpose,search_purpose_index,avg_PAG_VIEW_CT
0,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3.0,0,0.000000,107.0
1,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,이솝,3.0,0,0.000000,107.0
2,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다 스무드,1.0,1,0.285714,107.0
3,4142395,6158159,85,179538,notset,아베다,39000,1,단독6월라이트엘리먼츠헤어에센스세트,퍼스널케어,헤어케어,헤어케어선물세트,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,아베다,23.0,1,1.000000,107.0
4,4142395,6158159,23,558739,상품명이솝제라늄리프바디클렌저,이솝,56000,1,이솝제라늄리프바디클렌저500ml이솝제라늄리프바디클렌저,퍼스널케어,바디케어,바디워시,F,30.0,0,1,2018-06-19,107,6524,desktop,Seoul,Seoul,6,19,1,입생로랑,3.0,0,0.000000,107.0


In [36]:
print(test[test["search_purpose"]==0]["avg_PAG_VIEW_CT"].mean())
print(test[test["search_purpose"]==1]["avg_PAG_VIEW_CT"].mean())

198.1626877498588
171.15984599868096


In [35]:
test_check = test.groupby(["CLNT_ID", "SESS_ID"])["search_purpose", "avg_PAG_VIEW_CT"].mean()

print(test_check[test_check["search_purpose"]<=0.5]["avg_PAG_VIEW_CT"].mean())
print(test_check[test_check["search_purpose"]>0.5]["avg_PAG_VIEW_CT"].mean())

140.27166360522452
109.10792525633042


### think3. 날씨 데이터 결합하여 구매 패턴 분석

    - 챗봇에 입력할 변수로 날씨 이용
    
    - 2018.4.1. ~ 2018.9.30.
    
    - ZON_NM으로 분류할 수는 있으나, CITY_NM으로 분류할 수는 없음(데이터 없는 곳 있음)
        : 도별 날씨는 각 도에 속한 지역의 최빈값으로 대체 가능
        
    - https://www.weatheri.co.kr/bygone/bygone01.php 에서 크롤링 필요

In [9]:
final_data["ZON_NM"].unique()

array(['Seoul', 'Gyeonggi-do', 'Gangwon-do', 'Gyeongsangbuk-do', 'Daegu',
       'Busan', 'Chungcheongnam-do', 'Gwangju', 'Daejeon', 'Jeollabuk-do',
       'Gyeongsangnam-do', 'Ulsan', 'Jeollanam-do', 'Chungcheongbuk-do',
       'Incheon', 'Jeju-do'], dtype=object)

In [10]:
final_data["CITY_NM"].unique()

array(['Seoul', 'Suwon-si', 'Pyeongchang-gun', 'Gyeongju-si', 'Daegu',
       'Namyangju-si', 'Goyang-si', 'Gimpo-si', 'Busan', 'Cheonan-si',
       'Gwangju', 'Yongin-si', 'Daejeon', 'Jeonju-si', 'Gunsan-si',
       'Yangsan-si', 'Bucheon-si', 'Gwangmyeong-si', 'Ansan-si',
       'Gumi-si', 'Ulsan', 'Seongnam-si', 'Mungyeong-si', 'Gwangyang-si',
       'Hwaseong-si', 'Hanam-si', 'Gyeongsan-si', 'Paju-si',
       'Cheongwon-gun', 'Imsil-gun', 'Bonghwa-gun', 'Gimhae-si',
       'Osan-si', 'Iksan', 'Anyang', 'Pyeongtaek-si', 'Muan-gun',
       'Gongju-si', 'Haman-gun', 'Incheon', 'Suncheon-si', 'Chilgok-gun',
       'Cheongju-si', 'Seosan-si', 'Gunpo-si', 'Dangjin-si',
       'Yeongcheon-si', 'Pohang-si', 'Uijeongbu-si', 'Uiwang-si',
       'Hoengseong-gun', 'Chuncheon-si', 'Yeoncheon-gun', 'Asan-si',
       'Hamyang-gun', 'Yangju-si', 'Gangjin-gun', 'Jinju-si',
       'Donghae-si', 'Gwangju-si', 'Wonju-si', 'Hongcheon-gun', 'Jeju-si',
       'Siheung-si', 'Mokpo-si', 'Icheon-si', 'Gwach